In [5]:
import os
import re
import glob
import csv
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

load_dotenv()

usuario_db = os.getenv('POSTGRES_USER')
senha_db = os.getenv('POSTGRES_PASSWORD')
host_db = os.getenv('POSTGRES_HOST', 'localhost')
porta_db = os.getenv('POSTGRES_PORT', '5432')
nome_banco_db = os.getenv('POSTGRES_DB')

if not all([usuario_db, senha_db, nome_banco_db]):
    raise ValueError("Algumas variáveis de ambiente não foram encontradas no .env")

DATABASE_URL = f'postgresql://{usuario_db}:{senha_db}@{host_db}:{porta_db}/{nome_banco_db}'
engine = create_engine(DATABASE_URL)

print("Conexão criada.")

Conexão criada.


In [6]:
nome_schema_elt = 'staging'

with engine.begin() as connection:
    connection.execute(text(f"CREATE SCHEMA IF NOT EXISTS {nome_schema_elt};"))

print(f"Schema '{nome_schema_elt}' verificado.")

Schema 'staging' verificado.


In [7]:
print("Iniciando carga dos dados BRUTOS...")

caminho_pasta = '../data'
arquivos_csv = glob.glob(os.path.join(caminho_pasta, '*.csv'))

for arquivo in arquivos_csv:
    try:
        nome_arquivo = os.path.basename(arquivo)
        print(f"Lendo o arquivo: {nome_arquivo}")

        match_ano = re.search(r'(\d{4})', nome_arquivo)
        if match_ano:
            ano = match_ano.group(1)
            nome_tabela = f"raw_servidores_{ano}"
        else:
            nome_tabela = "raw_" + os.path.splitext(nome_arquivo)[0].lower().replace(' ', '_')

        df_raw = pd.read_csv(
            arquivo,
            sep=',',
            quoting=csv.QUOTE_NONE,
            engine='python',
            on_bad_lines='skip',
            encoding='utf-8',
            dtype=str
        )

        col_primeira = df_raw.columns[0]
        col_ultima = df_raw.columns[-1]

        df_raw[col_primeira] = df_raw[col_primeira].str.replace('"', '')
        df_raw[col_ultima] = df_raw[col_ultima].str.replace('"', '')
        df_raw.columns = df_raw.columns.str.replace('"', '')

        print(f"Carregando {len(df_raw)} linhas na tabela: {nome_schema_elt}.{nome_tabela}...")

        df_raw.to_sql(
            nome_tabela,
            con=engine,
            if_exists='replace',
            index=False,
            schema=nome_schema_elt
        )
        print(f"Tabela {nome_tabela} carregada com sucesso.")

    except Exception as e:
        print(f"Erro ao processar {nome_arquivo}: {e}")

print("\nProcesso de extração e carga concluído.")

Iniciando carga dos dados BRUTOS...
Lendo o arquivo: func2019_completo.csv
Carregando 462500 linhas na tabela: staging.raw_servidores_2019...
Tabela raw_servidores_2019 carregada com sucesso.
Lendo o arquivo: func2020_completo.csv
Carregando 465209 linhas na tabela: staging.raw_servidores_2020...
Tabela raw_servidores_2020 carregada com sucesso.
Lendo o arquivo: func2021_completo.csv
Carregando 470623 linhas na tabela: staging.raw_servidores_2021...
Tabela raw_servidores_2021 carregada com sucesso.

Processo de extração e carga concluído.
